# Collaborative Model Building

In [14]:
import pandas as pd
import numpy as np
import sagemaker_datawrangler
import matplotlib.pyplot as plt

In [ ]:
# Download Parquet from S3 locally, or use awswrangler to read pandas df directly from S3
no2_file = "YOUR-FILE-LOCATION"
weather_file = "YOUR-FILE-LOCATION"

In [15]:
df_no2 = pd.read_parquet(no2_file)
df_weather = pd.read_parquet(weather_file)

In [16]:
df_weather

In [7]:
# Pandas code generated by sagemaker_datawrangler
output_df = df_weather.copy(deep=True)


# Code to Replace with median for column: visibility to resolve warning: Missing values 
output_df['visibility']=output_df['visibility'].fillna(output_df['visibility'].median(skipna=True))


In [8]:
df_no2['ymd'] = pd.to_datetime(df_no2['ymd'], infer_datetime_format=True)
df_no2 = df_no2.set_index('ymd')

idx = pd.date_range(df_no2.index.min(), df_no2.index.max())
df_no2 = df_no2.reindex(idx, fill_value=None)
df_no2 = df_no2.interpolate(method='time')

In [9]:
output_df['date'] = pd.to_datetime(output_df['date'], infer_datetime_format=True)
output_df = output_df.set_index('date').sort_index()

In [10]:
min_viable_date = max(df_no2.index.min(), output_df.index.min())
max_viable_date = min(df_no2.index.max(), output_df.index.max())

In [11]:
print(f"Merging dataframes between {min_viable_date} and {max_viable_date}")

comp_df = pd.merge(
    output_df[min_viable_date:max_viable_date],
    df_no2[min_viable_date:max_viable_date][['no2_avg']],
    left_index=True, right_index=True
)

Merging dataframes between 2016-03-06 00:00:00 and 2022-11-14 00:00:00


In [12]:
mydata = comp_df[['wind_avg','no2_avg']]

x = mydata['wind_avg']
y = mydata['no2_avg']
plt.scatter(x, y)

z = np.polyfit(x, y, 1)
p = np.poly1d(z)
plt.plot(x,p(x),"r--")

plt.ylabel('NO2 Conc. ppm')
plt.xlabel('Wind Speed (mph)')
plt.show()

In [13]:
# Drop the 1st row as NaN
aq_df = comp_df.iloc[1:].copy()

# Drop visibility as it didn't seem correlate much and has NaNs that break the training
aq_df = aq_df.drop('visibility', 1)

# Use the data from years 2016 up to 2020 as training, and the year 2021 as our candidate year for testing and validating our model.
aq_train_df = aq_df[aq_df.index.year < 2021]
aq_test_df = aq_df[aq_df.index.year == 2021]

x_train_df = aq_train_df.drop('no2_avg',1)
x_train = x_train_df.values.astype('float32')


x_test_df = aq_test_df.drop('no2_avg',1)
x_test = x_test_df.values.astype('float32')

y_train_df = aq_train_df[["no2_avg"]]
y_train = y_train_df.values[:, 0].astype('float32')

y_test_df = aq_test_df[["no2_avg"]]
y_test = y_test_df.values[:, 0].astype('float32')

In [14]:
from math import sqrt
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score

def smape(actual, predicted):
    dividend= np.abs(np.array(actual) - np.array(predicted))
    denominator = np.array(actual) + np.array(predicted)
    
    return 2 * np.mean(np.divide(dividend, denominator, out=np.zeros_like(dividend), where=denominator!=0, casting='unsafe'))

def print_metrics(y_test, y_pred):
    print("RMSE: %.4f" % sqrt(mean_squared_error(y_test, y_pred)))
    print('Variance score: %.4f' % r2_score(y_test, y_pred))
    print('Explained variance score: %.4f' % explained_variance_score(y_test, y_pred))
    forecast_err = np.array(y_test) - np.array(y_pred)
    print('Forecast bias: %.4f' % (np.sum(forecast_err) * 1.0/len(y_pred) ))
    print('sMAPE: %.4f' % smape(y_test, y_pred))

In [15]:
from sklearn.linear_model import LinearRegression

In [17]:
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor(max_depth=4)
reg.fit(x_train, y_train)
reg.score(x_test, y_test)


# Real Time Collaboration - Model Improvement

In [4]:
# from sklearn.ensemble import RandomForestRegressor
# reg = RandomForestRegressor(max_depth=4)
# reg.fit(x_train, y_train)
# reg.score(x_test, y_test)

In [ ]:
result = reg.predict(x_test)
print_metrics(y_test, result)

In [ ]:
y_pred_df = pd.DataFrame(result, columns=y_test_df.columns).set_index(y_test_df.index).sort_index()

plt.plot(y_test_df, label='actual')
plt.plot(y_pred_df, label='forecast')
plt.legend()
plt.show()


In [ ]:
import joblib

# Save model
joblib.dump(reg, "data/model.pkl") 

In [62]:
# Upload to S3
!aws s3 cp data/model.pkl s3://YOUR-S3-BUCKET/airquality-experiment/